Grab fablib manager

In [1]:
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress
import json
import shlex
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Credential Manager,cm.fabric-testbed.net
Orchestrator,orchestrator.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
Bastion Username,sjzhu_0000105651
Bastion Private Key File,/home/fabric/work/fabric_config/fabric_bastion_key
Bastion Host,bastion.fabric-testbed.net
Bastion Private Key Passphrase,None
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Slice Private Key Passphrase,None


Find a site

In [2]:
# find a site with appropriate cores
cores_per_node = 8
ram_per_node = 32
disk_per_node = 20 # we will modify this later to give fileservers more
site1 = fablib.get_random_site(
    filter_function=lambda x: x["cores_available"] > cores_per_node * 3 and x["ram_available"] > ram_per_node * 3 and x["disk_available"] > disk_per_node * 3
)
fablib.show_site(site1, 'json')

{
    "name": "STAR",
    "state": "Active",
    "address": "710 N Lake Shore Dr, Chicago, IL 60611",
    "location": [
        41.89537135,
        -87.61663220067463
    ],
    "hosts": 6,
    "cpus": 12,
    "cores_available": 552,
    "cores_capacity": 768,
    "cores_allocated": 216,
    "ram_available": 2104,
    "ram_capacity": 3012,
    "ram_allocated": 908,
    "disk_available": 108316,
    "disk_capacity": 109696,
    "disk_allocated": 1380,
    "nic_basic_available": 675,
    "nic_basic_capacity": 762,
    "nic_basic_allocated": 87,
    "nic_connectx_6_available": 2,
    "nic_connectx_6_capacity": 2,
    "nic_connectx_6_allocated": 0,
    "nic_connectx_5_available": 5,
    "nic_connectx_5_capacity": 6,
    "nic_connectx_5_allocated": 1,
    "nvme_available": 18,
    "nvme_capacity": 20,
    "nvme_allocated": 2,
    "tesla_t4_available": 4,
    "tesla_t4_capacity": 6,
    "tesla_t4_allocated": 2,
    "rtx6000_available": 3,
    "rtx6000_capacity": 6,
    "rtx6000_allocated": 

'{\n    "name": "STAR",\n    "state": "Active",\n    "address": "710 N Lake Shore Dr, Chicago, IL 60611",\n    "location": [\n        41.89537135,\n        -87.61663220067463\n    ],\n    "hosts": 6,\n    "cpus": 12,\n    "cores_available": 552,\n    "cores_capacity": 768,\n    "cores_allocated": 216,\n    "ram_available": 2104,\n    "ram_capacity": 3012,\n    "ram_allocated": 908,\n    "disk_available": 108316,\n    "disk_capacity": 109696,\n    "disk_allocated": 1380,\n    "nic_basic_available": 675,\n    "nic_basic_capacity": 762,\n    "nic_basic_allocated": 87,\n    "nic_connectx_6_available": 2,\n    "nic_connectx_6_capacity": 2,\n    "nic_connectx_6_allocated": 0,\n    "nic_connectx_5_available": 5,\n    "nic_connectx_5_capacity": 6,\n    "nic_connectx_5_allocated": 1,\n    "nvme_available": 18,\n    "nvme_capacity": 20,\n    "nvme_allocated": 2,\n    "tesla_t4_available": 4,\n    "tesla_t4_capacity": 6,\n    "tesla_t4_allocated": 2,\n    "rtx6000_available": 3,\n    "rtx6000_cap

In [3]:
slice_name = 'ndn-poc-sjzhu2'

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

hosts = [f"{site1.lower()}-c1", f"{site1.lower()}-r1", f"{site1.lower()}-p1"]

In [4]:
# Define a subnet to use for the L2 Network
from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [5]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

for h in hosts:

    node = slice.add_node(name=f"{h}-a", cores=cores_per_node, ram=ram_per_node, disk=disk_per_node, site=site1, image='default_ubuntu_22')
    node.add_fabnet()
    node.add_post_boot_upload_directory("scripts")
    node.add_post_boot_execute("./scripts/install-ndn-dpdk.sh")

#Submit Slice Request
slice.submit();


Retry: 10, Time: 769 sec


ID,809020a7-5535-4efa-83f1-74d13f8ae7d2
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-13 17:31:01 +0000
Lease Start (UTC),2023-11-12 17:31:01 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
e7fb05e9-7bfa-4969-8bd4-f1273f1a4796,star-c1-a,8,32,100,default_ubuntu_22,qcow2,star-w2.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fecf:3ace,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fecf:3ace,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6195d921-514a-4630-a55e-e056a1126de5,star-p1-a,8,32,100,default_ubuntu_22,qcow2,star-w2.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe9c:b15f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe9c:b15f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
6bf15fd7-111a-4e83-b8ee-a15acaf9b74f,star-r1-a,8,32,100,default_ubuntu_22,qcow2,star-w2.fabric-testbed.net,STAR,ubuntu,2001:400:a100:3030:f816:3eff:fe11:5d5d,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2001:400:a100:3030:f816:3eff:fe11:5d5d,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
2631c461-4e70-45db-8421-8ce20f45e6de,FABNET_IPv4_STAR,L3,FABNetv4,STAR,10.129.130.0/24,10.129.130.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
star-c1-a-FABNET_IPv4_STAR_nic-p1,p1,star-c1-a,FABNET_IPv4_STAR,100,auto,,3A:A2:32:E1:20:6A,enp7s0,enp7s0,10.129.130.4,6
star-r1-a-FABNET_IPv4_STAR_nic-p1,p1,star-r1-a,FABNET_IPv4_STAR,100,auto,,3E:53:07:00:FA:62,enp7s0,enp7s0,10.129.130.2,6
star-p1-a-FABNET_IPv4_STAR_nic-p1,p1,star-p1-a,FABNET_IPv4_STAR,100,auto,,3E:54:C9:E2:00:33,enp7s0,enp7s0,10.129.130.3,6



Time to print interfaces 769 seconds


Here are all of the settings for the various 

In [17]:
FW_ARGS = {
  'mempool': {
    'DIRECT': { 'capacity': 524287, 'dataroom': 9146 },
    'INDIRECT': { 'capacity': 524287 },
  },
  'fib': {
    'capacity': 4095,
    'startDepth': 8,
  },
  'pcct': {
    'pcctCapacity': 65535,
    'csMemoryCapacity': 20000,
    'csIndirectCapacity': 20000,
  }
}
MEMIF_ARGS = {
    'scheme': "memif",
    'socketName': "/run/ndn/fileserver.sock",
    'id': 0,
    'role': "server",
    'dataroom': 9000
}
FS_ACTIVATE = {
    'eal': {
        'memPerNuma': {'0': 4*1024},
        'filePrefix': 'producer',
    },
    'mempool': {
        'DIRECT': {'capacity': 2**16-1, 'dataroom': 9200},
        'INDIRECT': {'capacity': 2**16-1},
        'PAYLOAD': {'capacity': 2**16-1, 'dataroom': 9200},
    },
    'face': {
        'scheme': 'memif',
        'socketName': MEMIF_ARGS['socketName'],
        'id': MEMIF_ARGS['id'],
        'dataroom': 9000,
        'role': 'client',
    },
    'fileServer': {
        'mounts': [
            {'prefix': "/producer/usr-local-share", 'path': "/usr/local/share"}
        ],
        'segmentLen': 6 * 1024,
    },
}

adj_list = {
    hosts[0]: [hosts[1]],
    hosts[1]: [hosts[0], hosts[2]],
    hosts[2]: [hosts[1]],
}

forwarder_hashes = {}
face_hashes = {}

In [18]:
# add eth ports to each node
for h in hosts:
    node = slice.get_node(f"{h}-a")
    output = node.execute(f'''
        echo {shlex.quote(json.dumps(FW_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ activate-forwarder
        ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-eth-port --netif enp7s0 --xdp --mtu 1500
        ''')
    forwarder_hashes[h] = json.loads(output[0].split("\n")[-2])

# add interface between nodes in adj_list
for h in adj_list:
    local_mac = forwarder_hashes[h]['macAddr']
    node = slice.get_node(f"{h}-a")
    for remote in adj_list[h]:
        remote_mac = forwarder_hashes[remote]['macAddr']
        output = node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-ether-face --local {local_mac} --remote {remote_mac}")
        face_hashes[f"{h}:{remote}"] = json.loads(output[0])

true
{"id":"ABRI1I05ERKD0","isDown":false,"macAddr":"3a:a2:32:e1:20:6a","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"NUG30KN077QJ8","isDown":false,"macAddr":"3e:53:07:00:fa:62","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
true
{"id":"L1N14OLKLQUJQ","isDown":false,"macAddr":"3e:54:c9:e2:00:33","mtu":1500,"name":"net_af_xdp_3","numaSocket":null}
{"id":"A7H2NQAQ6NID4JUM"}
{"id":"NIQJMSTVFRVJ90P3"}
{"id":"NIQJMSTVFFS310P0"}
{"id":"LDTHIGVBTUQ3IK0"}


In [19]:
print(forwarder_hashes)
print(face_hashes)
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
print(f"{hosts[0]}:{hosts[1]}")

{'star-c1': {'id': 'ABRI1I05ERKD0', 'isDown': False, 'macAddr': '3a:a2:32:e1:20:6a', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'star-r1': {'id': 'NUG30KN077QJ8', 'isDown': False, 'macAddr': '3e:53:07:00:fa:62', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}, 'star-p1': {'id': 'L1N14OLKLQUJQ', 'isDown': False, 'macAddr': '3e:54:c9:e2:00:33', 'mtu': 1500, 'name': 'net_af_xdp_3', 'numaSocket': None}}
{'star-c1:star-r1': {'id': 'A7H2NQAQ6NID4JUM'}, 'star-r1:star-c1': {'id': 'NIQJMSTVFRVJ90P3'}, 'star-r1:star-p1': {'id': 'NIQJMSTVFFS310P0'}, 'star-p1:star-r1': {'id': 'LDTHIGVBTUQ3IK0'}}
star-c1:star-r1


In [20]:
# add fileserver configuration to producer
producer_name = hosts[2] # hardcode this for now
producer_node = slice.get_node(f"{producer_name}-a")
output = producer_node.execute(f'''
    MEMIF_FACE=$(echo {shlex.quote(json.dumps(MEMIF_ARGS))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ create-face)
    ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh $(echo $MEMIF_FACE | jq -r .id)
    ''')
# start fileserver instance
output = producer_node.execute(f'''
    sudo ndndpdk-ctrl --gqlserver http://127.0.0.1:3031/ systemd start
    echo {shlex.quote(json.dumps(FS_ACTIVATE))} | ndndpdk-ctrl --gqlserver http://127.0.0.1:3031 activate-fileserver
    ''')

{"id":"LDPHGOTVLJQN8N7KB5D77UDCEN69CP0"}
true


In [21]:
# add FIB entries to point to producer
c_node = slice.get_node(f"{hosts[0]}-a")
c_to_r = face_hashes[f"{hosts[0]}:{hosts[1]}"]
output = c_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {c_to_r['id']}")

r_node = slice.get_node(f"{hosts[1]}-a")
r_to_p = face_hashes[f"{hosts[1]}:{hosts[2]}"]
r_node.execute(f"ndndpdk-ctrl --gqlserver http://127.0.0.1:3030/ insert-fib --name /producer --nh {r_to_p['id']}")

{"id":"A7L2LI8EEIG9IGVB50792US58786K8G"}
{"id":"NIUJKKVB7EUNR08RS3FBLERIT41OMM0"}


('{"id":"NIUJKKVB7EUNR08RS3FBLERIT41OMM0"}\n', '')

In [13]:
from datetime import datetime
from datetime import timezone
from datetime import timedelta

end_date = (datetime.now(timezone.utc) + timedelta(days=4)).strftime("%Y-%m-%d %H:%M:%S %z")
try:
    slice = fablib.get_slice(slice_name)
    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

In [14]:
slice.update()

In [15]:
slice.show()

ID,809020a7-5535-4efa-83f1-74d13f8ae7d2
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-16 18:05:52 +0000
Lease Start (UTC),2023-11-12 17:31:01 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


ID,809020a7-5535-4efa-83f1-74d13f8ae7d2
Name,ndn-poc-sjzhu2
Lease Expiration (UTC),2023-11-16 18:05:52 +0000
Lease Start (UTC),2023-11-12 17:31:01 +0000
Project ID,6ce270de-788d-4e07-8bae-3206860a6387
State,StableOK


TODO:
- script forwarder setup
  - use example activation parameters, excluding the CPU cores
- script forwarders to connect to each other
  - will need mac address from each and then create ether-face between pairs
- script fileserver setup
  - use example activation parameters, possibly editing the mountpoints
- use fileserver docs to setup FIB to fileserver using /producerN name
- create FIB entries for fileservers in each forwarder